#Configuración de ambiente

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Instalamos polygon api pra extraer datos financieros
# Instalamos bar_chart_race para la creación de graficos de carreras de barras
# Instalamos ffmpeg para poder guardar los graficos de carrera de barras en formato de video

!pip install polygon-api-client bar_chart_race ffmpeg

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 16.3 MB/s eta 0:00:00
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=c5dc20e7a14d08cbc445b32e4f666d6c66360c2a102dca655cae7c917d1d4d57
  Stored in directory: /root/.cache/pip/wheels/56/30/c5/576bdd729f3bc062d62a551be7fefd6ed2f761901568171e4e
Successfully built ffmpeg
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os
from google.colab import userdata
from polygon import RESTClient
from datetime import datetime
import bar_chart_race as bcr
import warnings

In [11]:
# Versiones al momento de la ejecución y versiones usadas al momento de la creación de este codigo

!python -V  # Python 3.11.13
print('------')
!pip show Pandas | grep 'Name\|Version'  # pandas Version: 2.2.2
print('------')
!pip show Numpy | grep 'Name\|Version'   # numpy Version: 2.0.2
print('------')
!pip show Matplotlib | grep 'Name\|Version'  # matplotlib Version: 3.10.0
print('------')
!pip show plotly | grep 'Name\|Version'  # plotly Version: 5.24.1
print('------')
!pip show seaborn | grep 'Name\|Version' # seaborn Version: 0.13.2
print('------')
!pip show polygon-api-client | grep 'Name\|Version'  # polygon-api-client Version: 1.15.3
print('------')
!pip show bar_chart_race | grep 'Name\|Version'  # bar-chart-race Version: 0.1.0
print('------')
!pip show ffmpeg | grep 'Name\|Version'  # ffmpeg Version: 1.4
print('------')


Python 3.11.13
------
Name: pandas
Version: 2.2.2
------
Name: numpy
Version: 2.0.2
Name: lapack-lite
Name: dragon4
Name: libdivide
Name: Meson
Name: spin
Name: OpenBLAS
Name: LAPACK
Name: GCC runtime library
Version 3.1, 31 March 2009
                       Version 3, 29 June 2007
  5. Conveying Modified Source Versions.
  14. Revised Versions of this License.
Name: libquadmath
------
Name: matplotlib
Version: 3.10.0
------
Name: plotly
Version: 5.24.1
------
Name: seaborn
Version: 0.13.2
------
Name: polygon-api-client
Version: 1.15.3
------
Name: bar-chart-race
Version: 0.1.0
------
Name: ffmpeg
Version: 1.4
------


#Obtención de datos

##Importando datos (Forbes Global 2008-2023 / Fortune Global 500 / Continentes y Paises)

In [ ]:
carpeta_forbes_global = '/content/drive/MyDrive/Alura/Challenge Data Insider/aroissues-forbes-global-2000-2008-2019/'
carpeta_continentes_paises = '/content/drive/MyDrive/Alura/Challenge Data Insider/continentes_y_paises/'
carpeta_fortune_global = '/content/drive/MyDrive/Alura/Challenge Data Insider/fortune_global_500/'


def leer_archivos(carpeta:str):

    """
    La funcion obtiene la direccion de una carpeta e importa todos los datos a una lista de dataframes

    arg:
        carpeta (str): direccion de la carpeta donde se encuentran los archivos

    return:
        datos (list): lista de dataframes con los datos importados

    raise:
        Exception: si ocurre un error al importar el archivo
        seguramente sean excepciones relacionadas con la funcion
        pd.read_html() (ValueError, FileNotFoundError, urllib.error.URLError,
        urllib.error.URLError)

    """
    datos = []
    for archivo in os.listdir(carpeta):
        ruta_completa = os.path.join(carpeta, archivo)

        if os.path.isfile(ruta_completa):
          try:
            if archivo.endswith('.csv'):
              df = pd.read_csv(ruta_completa)
              datos.append(df)
              print(f"Archivo {archivo} importado correctamente.")
            elif archivo.endswith('.xlsx'):
              df = pd.read_excel(ruta_completa)
              datos.append(df)
              print(f"Archivo {archivo} importado correctamente.")
            else:
              df = pd.read_html(ruta_completa)
              df = pd.DataFrame(df[0])
              datos.append(df)
              print(f"Archivo {archivo} importado correctamente.")
          except Exception as e:
            print(f"Error al importar el archivo {archivo}: {e}")
    return datos


In [ ]:
datos_forbes_global = leer_archivos(carpeta_forbes_global)
datos_continentes_paises = leer_archivos(carpeta_continentes_paises)
datos_fortune = leer_archivos(carpeta_fortune_global)

##Importando y exportando datos Polygon

In [ ]:
# Obtenemos la llave para acceder a la API
client = RESTClient(userdata.get('POLYGON_API_KEY'))

# Define una lista de tickers de las empresas
tickers_empresas = ["NTDOF", "EA", "KONMY", "SQNXF", "UBSFF"]

# Crea un diccionario para almacenar los datos
datos_historicos_estructurados = {}

# Itera sobre los tickers y obtén los datos históricos
for ticker in tickers_empresas:
  try:
    # Obtiene los datos agregados diarios para el rango de fechas deseado
    resp = client.get_aggs(ticker, 1, "day", "2024-01-01", "2024-12-31")

    # Procesa la respuesta para extraer los datos específicos
    datos_empresa = []
    if resp: # Verifica si la respuesta no está vacía
      for agg in resp:
        # Convierte el timestamp a un formato de fecha legible (Polygon obtiene los datos de fecha en milisegundos)
        fecha = datetime.fromtimestamp(agg.timestamp / 1000)
        fecha = fecha.strftime('%d-%m-%Y')
        # Ahora la fecha se almacena como dato tipo object, entonces la convertimos a un dato tipo datetime
        fecha = pd.to_datetime(fecha, format='%d-%m-%Y')
        # Generamos una lista que almacenará los datos de cada ticker en forma de diccionario
        datos_empresa.append({
            "Symbol": ticker,
            "Fecha": fecha,
            "Precio_Apertura": agg.open,
            "Precio_Cierre": agg.close,
            "Precio_Maximo": agg.high,
            "Precio_Minimo": agg.low
        })
    # Se almacenan los datos en el diccionario
    datos_historicos_estructurados[ticker] = datos_empresa
    print(f"Datos históricos de {ticker} obtenidos exitosamente.")
  except Exception as e:
    print(f"Error al obtener y procesar datos para {ticker}: {e}")

In [ ]:
print(datos_historicos_estructurados['NTDOF'])

In [ ]:
# Convertimos los datos de cada empresa a dataframes

df_nintendo = pd.DataFrame(datos_historicos_estructurados['NTDOF'])
df_ea = pd.DataFrame(datos_historicos_estructurados['EA'])
df_konami = pd.DataFrame(datos_historicos_estructurados['KONMY'])
df_squarenix = pd.DataFrame(datos_historicos_estructurados['SQNXF'])
df_ubisoft = pd.DataFrame(datos_historicos_estructurados['UBSFF'])


In [ ]:
df_ubisoft.head()

In [ ]:
# Creamos un dataframe general con todos los datos

df_polygon = pd.concat([df_nintendo, df_ea, df_konami, df_squarenix, df_ubisoft])

df_polygon.to_csv('datos_polygon.csv', index=False)

In [ ]:
display(df_polygon.head(),df_polygon.info())

In [ ]:
# Exportamos los datos en formato .csv

df_nintendo.to_csv('datos_nintendo_2024.csv', index=False)
df_ea.to_csv('datos_ea_2024.csv', index=False)
df_konami.to_csv('datos_konami_2024.csv', index=False)
df_squarenix.to_csv('datos_squarenix_2024.csv', index=False)
df_ubisoft.to_csv('datos_ubisoft_2024.csv', index=False)

#Data Wrangling

##Datos Forbes

In [ ]:
# iteramos en la lista de dataframes y renombramos las columnas

for df in datos_forbes_global:
  df.rename(columns={'Company': 'Empresa', 'Industry':'Industria',
                     'Country':'Pais', 'Sales':'Ingresos', 'Profits':'Ganancias',
                     'Assets':'Activos', 'Market_Value':'Valor_de_mercado'}, inplace=True)

# Añadimos 2 nuevas columnas
  df['Margen_de_rentabilidad (%)'] = round((df['Ganancias'] / df['Ingresos'])*100,2)
  df['ROA (%)'] = round((df['Ganancias'] / df['Activos'])*100,2)


In [ ]:
# Añadimos la columna año, siguiendo el orden en el que se importaron
# lo hacemos de esta forma porque no se importaron con un orden especifico
# de lo contrario, usariamos un ciclo for

datos_forbes_global[0]['Ano'] = 2009
datos_forbes_global[1]['Ano'] = 2008
datos_forbes_global[2]['Ano'] = 2010
datos_forbes_global[3]['Ano'] = 2011
datos_forbes_global[4]['Ano'] = 2014
datos_forbes_global[5]['Ano'] = 2015
datos_forbes_global[6]['Ano'] = 2013
datos_forbes_global[7]['Ano'] = 2012
datos_forbes_global[8]['Ano'] = 2016
datos_forbes_global[9]['Ano'] = 2022
datos_forbes_global[10]['Ano'] = 2021
datos_forbes_global[11]['Ano'] = 2020
datos_forbes_global[12]['Ano'] = 2019
datos_forbes_global[13]['Ano'] = 2018
datos_forbes_global[14]['Ano'] = 2023
datos_forbes_global[15]['Ano'] = 2017

In [ ]:
datos_forbes_global[0].head()

##Datos Fortune Global 500

In [ ]:
# Imprimimos los nombres de las columnas de cada dataframe
# Esto porque hay inconsistencias en los nombres de las columnas
# De esta forma podemos identificar algún elemento común para agilizar
# el proceso de eliminación de columnas
for df in datos_fortune:
  print(df.columns)

In [ ]:
# Renombramos las columnas que deseamos mantener y eliminamos el resto

for df in datos_fortune:
  df.rename(columns={'RemoveName':'Empresa', 'RemoveEmployees':'Empleados'}, inplace=True)
  df.drop([columna for columna in df.columns if 'Remove' in columna], inplace=True, axis=1)
datos_fortune[0].head()

In [ ]:
# Al igual que en el caso anterior, añadimos la columna año de forma manual

datos_fortune[0]['Ano'] = 2019
datos_fortune[1]['Ano'] = 2021
datos_fortune[2]['Ano'] = 2020
datos_fortune[3]['Ano'] = 2015
datos_fortune[4]['Ano'] = 2017
datos_fortune[5]['Ano'] = 2018
datos_fortune[6]['Ano'] = 2016
datos_fortune[7]['Ano'] = 2022

In [ ]:
datos_fortune[7].head()

##Datos Continente_Pais

In [ ]:
datos_continentes_paises[0].info()

In [ ]:
# Renombramos nombres de columnas

datos_continentes_paises[0].rename(columns={'Country':'Pais', 'Code':'Codigo'}, inplace=True)

In [ ]:
datos_continentes_paises[0].head()

In [ ]:
datos_continentes_paises[1].info()

In [ ]:
# Renombramos nombres de columnas

datos_continentes_paises[1].rename(columns={'Continent_Name':'Continente',
                                            'Three_Letter_Country_Code':'Codigo',
                                            'Country_Name':'Pais'}, inplace=True)
datos_continentes_paises[1].drop('Continent_Code', inplace=True, axis=1)

In [ ]:
datos_continentes_paises[1].head()

In [ ]:
datos_continentes_paises[1]['Continente'].unique()

#Junción de tablas y analisis exploratorio

## Junción y análisis continentes_paises


In [ ]:
# Unificamos los datos de la lista datos_continentes_paises en un solo dataframe

df_paises_continentes = pd.merge(datos_continentes_paises[0], datos_continentes_paises[1], how='outer', on='Codigo')


df_paises_continentes = df_paises_continentes[['Pais_x', 'Codigo', 'Continente']].rename(columns={'Pais_x': 'Pais'})


display(df_paises_continentes.info(), df_paises_continentes.sample(10))

In [ ]:
df_paises_continentes.loc[df_paises_continentes['Pais'].isnull()]

In [ ]:
#Verificamos los duplicados y evaluamos si debemos eliminarlos o no

df_paises_continentes.loc[df_paises_continentes['Codigo'].duplicated(keep=False)]

In [ ]:
# Eliminamos los duplicados para facilitar la operacion de llenado de datos nulos en la columna 'Pais'

df_paises_continentes.drop_duplicates(subset='Codigo', inplace=True)

In [ ]:
# Creamos un dataframe con los datos faltantes y los introducimos en el dataframe original usando la columna codigo como indice
paises_faltantes = {'Codigo': ['ALA', 'ANT', 'ATA', 'ATF', 'BES', 'BHS', 'BLM', 'BVT', 'CCK', 'CXR', 'ESH', 'GLP', 'GUF', 'HMD', 'IOT', 'MSR', 'MTQ', 'MYT', 'NFK', 'NRU', 'PCN', 'PSE', 'REU', 'SGS', 'SHN', 'SJM', 'TCA', 'TKL', 'UMI', 'VAT', 'VIR', 'WLF'],
        'Pais': ['Åland Islands', 'Antigua and Barbuda', 'Antarctica', 'French Southern Territories', 'Bonaire, Sint Eustatius and Saba', 'Bahamas (the)', 'Saint Barthélemy', 'Bouvet Island', 'Cocos (Keeling) Islands (the)', 'Christmas Island', 'Western Sahara',
                    'Guadeloupe', 'French Guiana', 'Heard Island and McDonald Islands', 'British Indian Ocean Territory (the)', 'Montserrat', 'Martinique', 'Mayotte', 'Norfolk Island', 'Nauru', 'Pitcairn', 'Palestine, State of', 'Réunion', 'South Georgia and the South Sandwich Islands',
                    'Saint Helena, Ascension and Tristan da Cunha', 'Svalbard and Jan Mayen', 'Turks and Caicos Islands (the)', 'Tokelau', 'United States Minor Outlying Islands (the)', 'Holy See (the)', 'Virgin Islands (U.S.)', 'Wallis and Futuna']}

df = pd.DataFrame(paises_faltantes)
df.set_index('Codigo', inplace=True)
df_paises_continentes.set_index('Codigo', inplace=True)

df_paises_continentes['Pais'] = df_paises_continentes['Pais'].fillna(df['Pais'])

df_paises_continentes.reset_index(inplace=True)
df_paises_continentes.info()

In [ ]:
# Observamos cuales son los valores nulos que faltan por solucionar
df_paises_continentes.loc[df_paises_continentes['Continente'].isnull()]

In [ ]:
# Cambiamos el codigo KSV, este ya no se usa, ahora se utiliza XKX

df_paises_continentes.loc[df_paises_continentes['Codigo'] == 'KSV', 'Continente'] = 'Europe'

# Eliminamos el resto de valores, pues no pertenecen a ningun país
df_paises_continentes.dropna(inplace=True)

df_paises_continentes.info()

## Junción y análisis forbes 2000

In [ ]:
# unificamos los dataframes Forbes 2000 en el periodo solicitado para trabajar las preguntas que debemos resolver

# Dataframe de los datos en el periodo 2015 - 2022
df_forbes_2015_2022 = pd.concat([df for df in datos_forbes_global if df['Ano'].iloc[0] >= 2015 and df['Ano'].iloc[0] <= 2022])

In [ ]:
df_forbes_2015_2022['Industria'].unique()

In [ ]:
df_forbes_2015_2022['Industria'].loc[df_forbes_2015_2022['Industria'] == 'Telecommunications services'] = 'Telecommunications Services'

In [ ]:
# Iniciamos el análisis exploratorio del dataset para buscar datos nulos, faltantes e incongruentes
display(df_forbes_2015_2022.info(), df_forbes_2015_2022.describe().round(2))

In [ ]:
#Se observan irregularidades en la relación de ganancias respecto a los ingresos, lo cual se refleja en el margen de rentabilidad

df_forbes_2015_2022.loc[df_forbes_2015_2022['Ganancias'] >= df_forbes_2015_2022['Ingresos']]

In [ ]:
# Eliminamos todos los registros con ingresos menores a las ganancias

indices_a_eliminar = df_forbes_2015_2022.loc[df_forbes_2015_2022['Ganancias'] >= df_forbes_2015_2022['Ingresos']].index

df_forbes_2015_2022.drop(index=indices_a_eliminar, inplace=True)

In [ ]:
df_forbes_2015_2022.describe().round(2)

In [ ]:
# Unificamos el dataframe con los datos de continentes y paises

df_forbes_2015_2022 = pd.merge(df_forbes_2015_2022, df_paises_continentes, 'left', on='Pais')

In [ ]:
df_forbes_2015_2022.info()

In [ ]:
#observamos que paises tienen valores nulos en las columnas codigo o continente

display(df_forbes_2015_2022['Pais'].loc[df_forbes_2015_2022['Codigo'].isna()].unique())
display(df_forbes_2015_2022['Pais'].loc[df_forbes_2015_2022['Continente'].isna()].unique())

In [ ]:
# Observamos una empresa y un país que no podemos identificar
display(df_forbes_2015_2022.loc[df_forbes_2015_2022['Pais'] == 5855])

In [ ]:
# Eliminamos los regisros incongruentes y llenamos los datos nulos para el país South Korea

df_forbes_2015_2022.loc[df_forbes_2015_2022['Pais'] == 'South Korea', 'Continente'] = 'Asia'
df_forbes_2015_2022.loc[df_forbes_2015_2022['Pais'] == 'South Korea', 'Codigo'] = 'KR'
indices_a_eliminar = df_forbes_2015_2022.loc[df_forbes_2015_2022['Pais'] == 5855].index

df_forbes_2015_2022.drop(index=indices_a_eliminar, inplace=True)

In [ ]:
df_forbes_2015_2022.info()

In [ ]:
# Ahora, habiendo resuelto el problema de datos incongruentes en los ingresos y ganancias,
# podemos llenar los valores nulos con el método más adecuado, en este caso llenamos los valores
# con el promedio de la columna de su respectiva empresa, para ello hacemos una
# agrupación por empresas en la columna de interes y usamos el método transform
# para que la serie resultante tenga las mismas dimensiones de la columna

df_forbes_2015_2022['Ingresos'] = df_forbes_2015_2022.groupby(['Empresa'])['Ingresos'].transform(lambda x: x.fillna(x.mean()))
df_forbes_2015_2022['Ganancias'] = df_forbes_2015_2022.groupby(['Empresa'])['Ganancias'].transform(lambda x: x.fillna(x.mean()))
df_forbes_2015_2022['Activos'] = df_forbes_2015_2022.groupby(['Empresa'])['Activos'].transform(lambda x: x.fillna(x.mean()))
df_forbes_2015_2022['Valor_de_mercado'] = df_forbes_2015_2022.groupby(['Empresa'])['Valor_de_mercado'].transform(lambda x: x.fillna(x.mean()))

#Si quedan datos nulos de una empresa, quiere decir que todos los datos de la columna en dicha empresa son nulos

#Llenamos los datos nulos en el margen de rentabilidad y el ROA

df_forbes_2015_2022.loc[df_forbes_2015_2022['Margen_de_rentabilidad (%)'].isnull(),
                               'Margen_de_rentabilidad (%)'] = df_forbes_2015_2022['Ganancias'] / df_forbes_2015_2022['Ingresos']

df_forbes_2015_2022.loc[df_forbes_2015_2022['ROA (%)'].isnull(),
                               'ROA (%)'] = df_forbes_2015_2022['Ganancias'] / df_forbes_2015_2022['Activos']

In [ ]:
#hacemos una lista con los indices de los registros que aun puedan contener valores nulos y los eliminamos

indices_a_eliminar = df_forbes_2015_2022.loc[(df_forbes_2015_2022['Ingresos'].isnull()) |
                                             (df_forbes_2015_2022['Ganancias'].isnull()) |
                                             (df_forbes_2015_2022['Activos'].isnull()) |
                                             (df_forbes_2015_2022['Valor_de_mercado'].isnull())].index

df_forbes_2015_2022.drop(index=indices_a_eliminar, inplace=True)

In [ ]:
df_forbes_2015_2022.info()

In [ ]:
#Exportamos el dataframe
df_forbes_2015_2022.to_csv('forbes_2015_2022.csv', index=False)

In [ ]:
# ya tratado el dataframe, creamos el dataframe en el periodo solicitado para el cuestionario A
df_forbes_2022 = df_forbes_2015_2022.loc[df_forbes_2015_2022['Ano'] == 2022]

In [ ]:
df_forbes_2022['Industria'].unique()

In [ ]:
display(df_forbes_2022.info(), df_forbes_2022.describe().round(2))

In [ ]:
df_forbes_2022.to_csv('forbes_2022.csv', index=False)

#Cuestionario A

Las siguientes preguntas se refieren al ranking Forbes 2000 del año 2022:

1. ¿Cuál es el Top 10 de países con más empresas en Forbes para el periodo
indicado?

2. ¿Cuál es el Top 4 de países con más empresas en el área de tecnología y de telecomunicaciones?

3. ¿Cuál fue el margen de pérdida de las 5 empresas que presentaron mayores perjuicios considerando el total de pérdidas registradas en la Industria de los Hoteles, Restaurantes y entretenimiento?

4. Considerando a la Industria Petrolera en Asia, ¿Cuál o cuáles empresas superaron en más del 20% su margen de rentabilidad?

5. ¿Cuáles fueron las empresas norteamericanas con el mayor porcentaje de rentabilidad por industria?

6. Excluyendo a la industria Bancaria, ¿Cuáles fueron las empresas europeas con  mayores pérdidas registradas por industria a nivel global?

7. ¿Cuál fue la distribución de Ingresos y Activos con respecto a las ganancias de los bancos cuyos activos no superan los 30000 millones de dólares?

8. Considerando el histórico de valores de las acciones en el año 2022 de las 5 empresas de tu elección (Paso 2), Indica ¿Cuál fue la mejor semana para comprar y cuál para vender respectivamente? (Se compra cuando el precio es bajo, y se vende cuando el precio es alto)

###¿Cuál es el Top 10 de países con más empresas en Forbes para el periodo indicado?

In [ ]:
top_paises_con_mas_empresas = df_forbes_2022.groupby(['Pais','Codigo'])['Empresa'].count().sort_values(ascending=False).head(10).reset_index()
display(top_paises_con_mas_empresas)


###¿Cuál es el Top 4 de países con más empresas en el área de tecnología y de telecomunicaciones?

In [ ]:
top_paises_empresas_tech = df_forbes_2022.loc[(df_forbes_2022['Industria'] == 'Telecommunications Services') |
                                               (df_forbes_2022['Industria'] == 'Technology Hardware & Equipment')].groupby(['Pais','Codigo'])[('Empresa')].count().sort_values(ascending=False).head(4).reset_index()
display(top_paises_empresas_tech)


###¿Cuál fue el margen de pérdida de las 5 empresas que presentaron mayores perjuicios considerando el total de pérdidas registradas en la Industria de los Hoteles, Restaurantes y entretenimiento?

In [ ]:
top_empresas_margen_perdida = df_forbes_2022.loc[df_forbes_2022['Industria'] == 'Hotels, Restaurants & Leisure' ].groupby(['Empresa'])['Margen_de_rentabilidad (%)'].mean().sort_values().head(5).reset_index()

display(top_empresas_margen_perdida)

###Considerando a la Industria Petrolera en Asia, ¿Cuál o cuáles empresas superaron en más del 20% su margen de rentabilidad?

In [ ]:
margen_rentabilidad_Asia = df_forbes_2022.loc[(df_forbes_2022['Industria'] == 'Oil & Gas Operations') &
                                                 (df_forbes_2022['Continente'] == 'Asia')].groupby(['Empresa'])['Margen_de_rentabilidad (%)'].mean().sort_values(ascending=False).head(7).reset_index()

display(margen_rentabilidad_Asia )

###¿Cuáles fueron las empresas norteamericanas con el mayor porcentaje de rentabilidad por industria?

In [ ]:
empresas_NA_top_rentabilidad = df_forbes_2022.loc[(df_forbes_2022['Continente'] == 'North America')].groupby(['Industria'])[['Empresa','Margen_de_rentabilidad (%)']].max().sort_values(ascending=False, by='Margen_de_rentabilidad (%)').reset_index()

display(empresas_NA_top_rentabilidad)

###Excluyendo a la industria Bancaria, ¿Cuáles fueron las empresas europeas con mayores pérdidas registradas por industria a nivel global?

In [ ]:
empresas_EU_top_perdidas = df_forbes_2022.loc[(df_forbes_2022['Continente'] == 'Europe') &
                                              (df_forbes_2022['Industria'] != 'Banking')].groupby('Industria')[['Empresa','Ganancias']].min().sort_values(by='Ganancias').reset_index()
display(empresas_EU_top_perdidas)

###¿Cuál fue la distribución de Ingresos y Activos con respecto a las ganancias de los bancos cuyos activos no superan los 300000 millones de dólares?

In [ ]:
bancos_dist_activos_ingresos = df_forbes_2022.loc[(df_forbes_2022['Industria'] == 'Banking') &
                                                  (df_forbes_2022['Activos'] <= 300000), ['Ingresos','Activos','Ganancias']]

display(bancos_dist_activos_ingresos)

###Considerando el histórico de valores de las acciones en el año 2022 de las 5 empresas de tu elección (Paso 2), Indica ¿Cuál fue la mejor semana para comprar y cuál para vender respectivamente? (Se compra cuando el precio es bajo, y se vende cuando el precio es alto)

In [ ]:
df_polygon.head()

In [ ]:
df_polygon['Semana'] = df_polygon['Fecha'].dt.isocalendar().week

df_polygon_semanal = df_polygon.groupby(['Symbol', 'Semana']).agg(
    Fecha=('Fecha', 'first'),
    Precio_Apertura_Promedio=('Precio_Apertura', 'mean'),
    Precio_Cierre_Promedio=('Precio_Cierre', 'mean'),
    Precio_Maximo_Semanal=('Precio_Maximo', 'max'),
    Precio_Minimo_Semanal=('Precio_Minimo', 'min')
).reset_index()

df_polygon_semanal.to_csv('polygon_semanal.csv', index=False)

display(df_polygon_semanal.head())

In [ ]:
mejor_semana_compra = df_polygon_semanal.loc[df_polygon_semanal.groupby('Symbol')['Precio_Cierre_Promedio'].idxmin()]

display(mejor_semana_compra)

In [ ]:
mejor_semana_venta = df_polygon_semanal.loc[df_polygon_semanal.groupby('Symbol')['Precio_Cierre_Promedio'].idxmax()]

display(mejor_semana_venta)

#Cuestionario B
Las siguientes preguntas se refieren al ranking Forbes 2000 considerando el periodo 2015 - 2022:

1. ¿Cuál es el Top 10 de países con más empresas en Forbes para el periodo indicado?

2. ¿Cuál es el Top 5 de países con más empresas en el área de tecnología y de telecomunicaciones?

3. ¿Cuál fue el margen de pérdida de las 10 empresas que presentaron mayores perjuicios considerando el total de pérdidas registradas en la Industria de los Hoteles, Restaurantes y entretenimiento?

4. Considerando a la Industria Petrolera en las Américas, ¿Cuál o cuáles empresas superaron en más del 20% la rentabilidad de sus activos?

5. ¿Cuáles fueron las empresas europeas con el mayor porcentaje de rentabilidad por industria?

6. ¿Cuáles fueron las 10 empresas norteamericanas con  mayores pérdidas registradas por industria a nivel global?

7. ¿Cuál fue la distribución de Ingresos y Activos con respecto a las ganancias de los bancos cuyos activos no superan los 15000 millones de dólares?

8. ¿Cuáles fueron las ganancias anuales de las principales empresas en la Industria automotriz?

###¿Cuál es el Top 10 de países con más empresas en Forbes para el periodo indicado?

In [ ]:
top_paises_empresas_15_22 = df_forbes_2015_2022.groupby(['Pais','Codigo'])['Empresa'].count().sort_values(ascending=False).head(10).reset_index()
display(top_paises_empresas_15_22)

###¿Cuál es el Top 5 de países con más empresas en el área de tecnología y de telecomunicaciones?

In [ ]:
top_paises_empresas_tech_15_22 = df_forbes_2015_2022.loc[(df_forbes_2015_2022['Industria'] == 'Telecommunications Services') |
                                                         (df_forbes_2015_2022['Industria'] == 'Technology Hardware & Equipment')].groupby(['Pais','Codigo'])['Empresa'].count().sort_values(ascending=False).head(5).reset_index()
display(top_paises_empresas_tech_15_22)

###¿Cuál fue el margen de pérdida de las 10 empresas que presentaron mayores perjuicios considerando el total de pérdidas registradas en la Industria de los Hoteles, Restaurantes y entretenimiento?

In [ ]:
top_perdidas_empresas_HRE = df_forbes_2015_2022.loc[df_forbes_2015_2022['Industria'] == 'Hotels, Restaurants & Leisure'].groupby(['Empresa'])['Ganancias'].sum().sort_values().head(10).reset_index()

display(top_perdidas_empresas_HRE)

###Considerando a la Industria Petrolera en las Américas, ¿Cuál o cuáles empresas superaron en más del 20% la rentabilidad de sus activos?

In [ ]:
top_empresas_ROA = df_forbes_2015_2022.loc[((df_forbes_2015_2022['Continente'] == 'North America') |
                                                            (df_forbes_2015_2022['Continente'] == 'South America')) &
                                                            ((df_forbes_2015_2022['Industria'] == 'Oil & Gas Operations'))].groupby(['Empresa'])['ROA (%)'].mean().round(2).sort_values(ascending=False).reset_index()

top_empresas_ROA = top_empresas_ROA.loc[top_empresas_ROA['ROA (%)'] >= 20]

display(top_empresas_ROA)

###¿Cuáles fueron las empresas europeas con el mayor porcentaje de rentabilidad por industria?

In [ ]:
empresas_EU_top_rentabilidad = df_forbes_2015_2022.loc[df_forbes_2015_2022['Continente'] == 'Europe'].groupby('Industria')[['Empresa','Margen_de_rentabilidad (%)']].max().sort_values(by='Margen_de_rentabilidad (%)',ascending=False).reset_index()

display(empresas_EU_top_rentabilidad)

###¿Cuáles fueron las 10 empresas norteamericanas con mayores pérdidas registradas por industria a nivel global?

In [ ]:
empresas_NA_top_perdidas = df_forbes_2015_2022.loc[df_forbes_2015_2022['Continente'] == 'North America'].groupby('Industria')[['Empresa','Ganancias']].min().sort_values(by='Ganancias').head(10).reset_index()

display(empresas_NA_top_perdidas)

###¿Cuál fue la distribución de Ingresos y Activos con respecto a las ganancias de los bancos cuyos activos no superan los 15000 millones de dólares?

In [ ]:
bancos_dist_activos_ingresos_15_22 = df_forbes_2015_2022.loc[(df_forbes_2015_2022['Industria'] == 'Banking') &
                                                  (df_forbes_2015_2022['Activos'] <= 150000), ['Ingresos','Activos','Ganancias']].reset_index(drop=True)

display(bancos_dist_activos_ingresos_15_22)

###¿Cuáles fueron las ganancias anuales de las principales empresas en la Industria automotriz?

In [ ]:
ganancias_anuales_empresas_automotriz = df_forbes_2015_2022.loc[df_forbes_2015_2022['Industria'] == 'Transportation'].groupby(['Ano','Empresa'])['Ganancias'].sum().reset_index()

display(ganancias_anuales_empresas_automotriz)

##Parte 3

In [ ]:
df_fortune_global = pd.concat([df for df in datos_fortune])

display(df_fortune_global.info(), df_fortune_global.head())

In [ ]:
df_forbes_empleados = pd.merge(df_forbes_2015_2022, df_fortune_global, how='inner', on=['Ano','Empresa'])

display(df_forbes_empleados.info(), df_forbes_empleados.head())

In [ ]:
df_forbes_empleados.to_csv('forbes_empleados.csv', index=False)

# Visualización de datos

##Graficos cuestionario A

In [ ]:
top_paises_con_mas_empresas.head()

In [ ]:
plt.figure(figsize=(15, 8))

ax = sns.barplot(x='Pais', y='Empresa',data=top_paises_con_mas_empresas,
                 hue='Pais', palette='inferno', legend=False)
plt.suptitle('Paises con mayor número de empresas Forbes Global', fontsize=18)
plt.title('Top 10 de países con más empresas', fontsize=16)
plt.xlabel('País', fontsize=14)
plt.xticks(fontsize=12, rotation=45)
plt.ylabel('Número de Empresas', fontsize=14)
ax.set_yticklabels([])
sns.despine(left=True,bottom=True)
ax.tick_params(left=False, bottom=False)

for i,j in enumerate(top_paises_con_mas_empresas['Empresa']):
    ax.text(i,j+10,str(j),color='black',fontsize=12,ha='center',va='center')


plt.show()

**Informe del Gráfico: Paises con mayor número de empresas Forbes Global (2022)**

Este gráfico de barras muestra el Top 10 de países con la mayor cantidad de empresas incluidas en el ranking Forbes Global del año 2022. Se observa que Estados Unidos lidera la lista con una cantidad significativamente mayor de empresas, seguido por China y Japón. Este gráfico resalta la concentración de grandes corporaciones a nivel global en estos países.

In [ ]:
top_paises_empresas_tech

In [ ]:
plt.figure(figsize=(15, 8))

ax = sns.barplot(x='Pais', y='Empresa', hue='Pais', data=top_paises_empresas_tech,
                 palette='inferno', legend=False)
plt.suptitle('Paises con mayor número de empresas en la industria\n tecnológica y telecomunicaciones en el año 2022',
             fontsize=18, y=1.01)
plt.title('Top 4 de países con más empresas Tech', fontsize=16)
plt.xlabel('País', fontsize=14)
plt.xticks(fontsize=12)
plt.ylabel('Cantidad de Empresas', fontsize=14)
for i,j in enumerate(top_paises_empresas_tech['Empresa']):
    ax.text(i,j+.5,str(j),color='black',fontsize=12,ha='center',va='center')

ax.set_yticklabels([])
sns.despine(left=True,bottom=True)
ax.tick_params(left=False, bottom=False)


plt.show()

**Informe del Gráfico: Paises con mayor número de empresas en la industria tecnológica y telecomunicaciones (2022)**

Este gráfico de barras presenta el Top 4 de países con la mayor cantidad de empresas en las áreas de tecnología y telecomunicaciones según el ranking Forbes Global 2022. Similar al ranking general, Estados Unidos y China ocupan los primeros lugares, lo que indica su dominio en estas industrias estratégicas. Japón y Taiwan también figuran en este top, mostrando su relevancia en el sector tecnológico.

In [ ]:
top_empresas_margen_perdida

In [ ]:
plt.figure(figsize=(15, 8))

ax = sns.barplot(x='Empresa', y='Margen_de_rentabilidad (%)', hue='Empresa',
                 data=top_empresas_margen_perdida, palette='Reds_r', legend=False)
plt.suptitle('Grafico del top paises con el mayor margen de perdida\n en la Industria de los Hoteles, Restaurantes y entretenimiento?',
             fontsize=18, y=1.02)
plt.title('Top 5 de países con mayor margen de perdida', fontsize=16)
plt.xlabel('Empresa', fontsize=14)
plt.xticks(fontsize=12)
plt.ylabel('Margen de Rentabilidad (%)', fontsize=14)
plt.ylim(-300,50)
ax.set_yticklabels([])
sns.despine(left=True,bottom=True)
plt.axhline(0, color='grey', linewidth=0.8)
for i,j in enumerate(top_empresas_margen_perdida['Margen_de_rentabilidad (%)']):
    ax.text(i,j-8 if j < 0 else j + 8,str(j) + '%',color='black',fontsize=12,ha='center',va='center')

ax.tick_params(left=False, bottom=False)


plt.show()

**Informe del Gráfico: Margen de Rentabilidad de las Empresas en la Industria de Hoteles, Restaurantes y Entretenimiento (2022)**

Este gráfico de barras muestra el margen de rentabilidad de las 5 empresas con mayores pérdidas en la industria de Hoteles, Restaurantes y Entretenimiento en el ranking Forbes Global 2022. Se observa que Carnival Corporation presentó el mayor margen de pérdida, seguido por Las Vegas Sands y Caesars Entertainment. Esto refleja el impacto negativo que enfrentaron estas empresas en el año 2022.

In [ ]:
margen_rentabilidad_Asia

In [ ]:
margen_rentabilidad_Asia.loc[margen_rentabilidad_Asia['Empresa'] == 'Saudi Arabian Oil Company (Saudi Aramco)', 'Empresa'] = 'Saudi Aramco'
display(margen_rentabilidad_Asia['Empresa'])

In [ ]:
plt.figure(figsize=(15, 8))

ax = sns.barplot(x='Empresa', y='Margen_de_rentabilidad (%)', hue='Empresa',
                 data=margen_rentabilidad_Asia, palette='inferno', legend=False)

plt.suptitle('Margen de Rentabilidad de la Industria Petrolera en Asia', fontsize=18)
plt.title('Empresas que superaron más del 20% su margen de rentabilidad', fontsize=16)
plt.xlabel('Empresa', fontsize=14)
plt.xticks(fontsize=12)
plt.ylabel('Margen de Rentabilidad (%)', fontsize=14)
plt.ylim(0,50)
ax.set_yticklabels([])
sns.despine(left=True,bottom=True)

for i,j in enumerate(margen_rentabilidad_Asia['Margen_de_rentabilidad (%)']):
    ax.text(i,j+1,str(j) + '%',color='black',fontsize=12,ha='center',va='center')

ax.tick_params(left=False, bottom=False)


plt.show()

**Informe del Gráfico: Margen de Rentabilidad de la Industria Petrolera en Asia (2022)**

Este gráfico de barras presenta las empresas de la industria petrolera en Asia que superaron en más del 20% su margen de rentabilidad según el ranking Forbes Global 2022. Empresas como Novatek, Surgutneftegas y CNOOC destacan con altos márgenes de rentabilidad, lo que sugiere un año favorable para estas compañías en el sector petrolero asiático.

In [ ]:
plt.figure(figsize=(15, 8))

# Define una paleta de colores personalizada
colores_personalizados = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
    '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5',
    '#c49c94', '#f7b6d2', '#c7c7c7', '#dbdb8d', '#9edae5',
    '#393b79', '#637939', '#8c6d31', '#843c39', '#7b4173',
    '#a55194'
]


ax = sns.barplot(x='Margen_de_rentabilidad (%)', y='Empresa', hue='Industria',
                 data=empresas_NA_top_rentabilidad, orient='h', palette=colores_personalizados)

plt.suptitle('Margen de Rentabilidad de las Empresas por Industria en Norte America (2022)', fontsize=18)
plt.title('Empresas con mayor margen de rentabilidad por industria', fontsize=16)
plt.xlabel('Margen_de_rentabilidad (%)', fontsize=14)
plt.ylabel('Empresa', fontsize=14)
plt.yticks(fontsize=12)
ax.set_xticklabels([])
sns.despine(left=True,bottom=True)
ax.legend(bbox_to_anchor=(1.2,-0.05), bbox_transform=ax.transAxes,
          fontsize=10, loc='lower right', title='Industria')

for i,j in enumerate(empresas_NA_top_rentabilidad['Margen_de_rentabilidad (%)']):
    ax.text(j+3,i,str(j) + '%',color='black',fontsize=12,ha='center',va='center')

ax.tick_params(left=False, bottom=False)


plt.show()

**Informe del Gráfico: Margen de Rentabilidad de las Empresas por Industria en Norte América (2022)**

Este gráfico de barras horizontal exhibe las empresas norteamericanas con el mayor porcentaje de rentabilidad por industria en el ranking Forbes Global 2022. Se puede observar la diversidad de industrias representadas y cómo empresas individuales dentro de cada sector lograron altos márgenes de rentabilidad en Norteamérica durante este año.

In [ ]:
empresas_EU_top_perdidas.head()

In [ ]:
plt.figure(figsize=(15, 8))

ax = sns.barplot(x='Ganancias', y='Empresa', hue='Industria',
                 data=empresas_EU_top_perdidas.head(), palette='Reds_r', orient='h')

plt.suptitle('Mayores perdidas generadas a nivel global por empresas europeas (2022)',
             fontsize=18)
plt.title('Empresas con mayores perdidas por industria (Excluyendo la bancaria)',
          fontsize=16)
plt.xlabel('Margen_de_rentabilidad (%)', fontsize=14)
plt.ylabel('Empresa', fontsize=14)
plt.yticks(fontsize=12)
plt.xlim(-12000,0)
ax.set_xticklabels([])
sns.despine(left=True,bottom=True)

ax.legend(bbox_to_anchor=(1.23,0.75), bbox_transform=ax.transAxes,
          fontsize=10, loc='lower right', title='Industria')

for i,j in enumerate(empresas_EU_top_perdidas['Ganancias'].head()):
    ax.text(j-450, i, str(j) + '$', color='black',fontsize=11,ha='center',va='center')

ax.tick_params(left=False, bottom=False)


plt.show()

**Informe del Gráfico: Mayores pérdidas generadas a nivel global por empresas europeas (2022)**

Este gráfico de barras horizontal ilustra las empresas europeas (excluyendo la industria bancaria) que registraron las mayores pérdidas a nivel global en el ranking Forbes Global 2022. BT Group en telecomunicaciones y Aena en transporte se destacan con las mayores pérdidas, lo que indica desafíos significativos en estos sectores para las empresas europeas.

In [ ]:
bancos_dist_activos_ingresos

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

sns.scatterplot(x='Ingresos', y='Activos', data=bancos_dist_activos_ingresos, size='Ganancias',  sizes=(50, 250), palette='inferno_r', hue='Ganancias')

plt.suptitle('Ingresos y Activos con respecto a las ganancias de los bancos (2022)', fontsize=18, y=1.02)
plt.title('Distribución de Ingresos y Activos con respecto a las ganancias\n cuyos activos no superan los 300000 millones de dólares', fontsize=16)
plt.xlabel('Ganancias', fontsize=14)
plt.ylabel('Activos', fontsize=14)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)

ax.legend(bbox_to_anchor=(1.2,-0.05), bbox_transform=ax.transAxes,
          fontsize=10, loc='lower right', title='Ganancias')

ax.tick_params(left=False, bottom=False)


plt.show()

**Informe del Gráfico: Distribución de Ingresos y Activos con respecto a las ganancias de los bancos (2022)**

Este gráfico de dispersión muestra la distribución de Ingresos y Activos en relación con las ganancias de los bancos con activos que no superan los 30,000 millones de dólares en el ranking Forbes Global 2022. El tamaño de los puntos representa los ingresos. Se puede observar la relación entre estas tres métricas para los bancos más pequeños.

In [ ]:
df_polygon.head()

In [ ]:
df_polygon_semanal.head()

In [ ]:
fig = px.line(df_polygon, x='Fecha', y='Precio_Cierre', color='Symbol',
              title=f'Precio de cierre diario de las acciones<br>{df_polygon["Symbol"].unique()}',
              labels={'Precio_Cierre': 'Precio de Cierre'}, width=1500, height=500)

fig.add_trace(go.Scatter(x=mejor_semana_venta['Fecha'], y=mejor_semana_venta['Precio_Cierre_Promedio'],
                         mode='markers', name='Mejor semana venta', marker=dict(size = 7,color='blue'),
                         hovertext=[f"Simbolo: {row['Symbol']}<br>Fecha: {row['Fecha'].strftime('%d-%m-%Y')}<br>Semana: {row['Semana']}<br>Precio: {row['Precio_Cierre_Promedio']:.2f}$"
                                    for index, row in mejor_semana_venta.iterrows()],
                         hoverinfo='text'))
fig.add_trace(go.Scatter(x=mejor_semana_compra['Fecha'], y=mejor_semana_compra['Precio_Cierre_Promedio'],
                         mode='markers', name='Mejor semana compra', marker=dict(size = 7,color='green'),
                         hovertext=[f"Simbolo: {row['Symbol']}<br>Fecha: {row['Fecha'].strftime('%d-%m-%Y')}<br>Semana: {row['Semana']}<br>Precio: {row['Precio_Cierre_Promedio']:.2f}$"
                                    for index, row in mejor_semana_compra.iterrows()],
                         hoverinfo='text'))

fig.show()

**Informe del Gráfico: Precio de cierre diario de las acciones (2024)**

Este gráfico de líneas interactivo muestra el precio de cierre diario de las acciones de las cinco empresas seleccionadas (NTDOF, EA, KONMY, SQNXF, UBSFF) a lo largo del año 2024. Se han añadido marcadores para indicar la "mejor semana para comprar" (precio mínimo semanal) en verde y la "mejor semana para vender" (precio máximo semanal) en azul para cada empresa, según los datos históricos de valores.

**Mejores Semanas para Comprar:**

EA: Semana 16, Fecha 2024-04-15, Precio de Cierre Promedio: 126.56<br>
KONMY: Semana 1, Fecha 2024-01-04, Precio de Cierre Promedio: 26.04<br>
NTDOF: Semana 17, Fecha 2024-04-22, Precio de Cierre Promedio: 48.29<br>
SQNXF: Semana 25, Fecha 2024-06-17, Precio de Cierre Promedio: 28.81<br>
UBSFF: Semana 49, Fecha 2024-12-02, Precio de Cierre Promedio: 12.23

**Mejores Semanas para Vender:**

EA: Semana 49, Fecha 2024-12-02, Precio de Cierre Promedio: 166.74<br>
KONMY: Semana 39, Fecha 2024-09-25, Precio de Cierre Promedio: 49.50<br>
NTDOF: Semana 50, Fecha 2024-12-09, Precio de Cierre Promedio: 60.45<br>
SQNXF: Semana 8, Fecha 2024-02-21, Precio de Cierre Promedio: 44.67<br>
UBSFF: Semana 7, Fecha 2024-02-16, Precio de Cierre Promedio: 26.09

##Graficos cuestionario B

In [ ]:
top_paises_empresas_15_22

In [ ]:
plt.figure(figsize=(15, 8))

ax = sns.barplot(x='Pais', y='Empresa', hue='Pais', data=top_paises_empresas_15_22,
                 palette='inferno', legend=False)

plt.suptitle('Cantidad de empresas por pais en el periodo 2015-2022', fontsize=18)
plt.title('Top 10 paises con más empresas', fontsize=16)
plt.xlabel('País', fontsize=14)
plt.xticks(fontsize=12, rotation=15)
plt.ylabel('Cantidad de empresas', fontsize=14)
ax.set_yticklabels([])
sns.despine(left=True,bottom=True)
for i,j in enumerate(top_paises_empresas_15_22['Empresa']):
    ax.text(i,j+75,str(j),color='black',fontsize=12,ha='center',va='center')

ax.tick_params(left=False, bottom=False)


plt.show()

**Informe del Gráfico: Cantidad de empresas por país en el periodo 2015-2022**

Este gráfico de barras presenta el Top 10 de países con la mayor cantidad de empresas incluidas en el ranking Forbes Global durante el periodo 2015-2022. De manera consistente con el año 2022, Estados Unidos, China y Japón mantienen su liderazgo en la concentración de grandes empresas a lo largo de estos años.

In [ ]:
top_paises_empresas_tech_15_22

In [ ]:
plt.figure(figsize=(15, 8))

ax = sns.barplot(x='Pais', y='Empresa', hue='Pais', data=top_paises_empresas_tech_15_22,
                 palette='inferno', legend=False)

plt.suptitle('Cantidad de empresas del Área de tecnología y telecomunicaciones\n por pais en el periodo 2015-2022', fontsize=18, y=1.01)
plt.title('Top 5 paises con más empresas Tech', fontsize=16)
plt.xlabel('País', fontsize=14)
plt.xticks(fontsize=12)
plt.ylabel('Cantidad de empresas', fontsize=14)
ax.set_yticklabels([])
sns.despine(left=True,bottom=True)
for i,j in enumerate(top_paises_empresas_tech_15_22['Empresa']):
    ax.text(i,j+2,str(j),color='black',fontsize=12,ha='center',va='center')

ax.tick_params(left=False, bottom=False)


plt.show()

**Informe del Gráfico: Cantidad de empresas del Área de tecnología y telecomunicaciones por país en el periodo 2015-2022**

Este gráfico de barras muestra el Top 5 de países con la mayor cantidad de empresas en las áreas de tecnología y telecomunicaciones en el ranking Forbes Global durante el periodo 2015-2022. Estados Unidos y China siguen siendo los principales actores, seguidos por Taiwan y Japón, lo que subraya su continua importancia en estos sectores a lo largo del tiempo.

In [ ]:
top_perdidas_empresas_HRE

In [ ]:
plt.figure(figsize=(15, 8))
colores = ['green' if v >= 0 else 'red' for v in top_perdidas_empresas_HRE['Ganancias']]

ax = sns.barplot(x='Empresa', y='Ganancias', hue='Empresa', data=top_perdidas_empresas_HRE, palette='coolwarm_r', legend=False)

plt.suptitle('Grafico del margen de perdida de las empresas del Hoteles, Restaurantes y Entretenimiento\n en el periodo 2015-2022', fontsize=18, y=1.01)
plt.title('Top 10 Empresas con mayor margen de perdidas', fontsize=16)
plt.xlabel('Empresa', fontsize=14)
plt.xticks(fontsize=12, rotation=15)
plt.ylabel('Margen de Rentabilidad', fontsize=14)
plt.ylim(-10000,2000)
plt.axhline(0, color='grey', linewidth=0.8)
ax.set_yticklabels([])
sns.despine(left=True,bottom=True)


for i,j in enumerate(top_perdidas_empresas_HRE['Ganancias']):
    ax.text(i,j-200 if j < 0 else j+200,str(j) + '$',color='black',fontsize=12,ha='center',va='center')

ax.tick_params(left=False, bottom=False)


plt.show()

**Informe del Gráfico: Margen de pérdida de las empresas del Hoteles, Restaurantes y Entretenimiento en el periodo 2015-2022**

Este gráfico de barras visualiza el margen de pérdida de las 10 empresas con mayores perjuicios en la industria de Hoteles, Restaurantes y Entretenimiento durante el periodo 2015-2022. Carnival Corporation y Royal Caribbean Group muestran las mayores pérdidas acumuladas en este sector a lo largo de los años indicados.

In [ ]:
top_empresas_ROA

In [ ]:
plt.figure(figsize=(15, 8))

ax = sns.barplot(x='Empresa', y='ROA (%)', data=top_empresas_ROA, hue='Empresa',
                 palette='inferno', legend=False)

plt.suptitle('Rentabilidad de activos en Empresas Americanas de\n la industria petrolera en el periodo 2015-2022', fontsize=18, y=1.01)
plt.title('Empresas que tienen un ROA superior al 20%', fontsize=16)
plt.xlabel('Empresa', fontsize=14)
plt.xticks(fontsize=12)
plt.ylabel('ROA (%)', fontsize=14)
ax.set_yticklabels([])
sns.despine(left=True,bottom=True)

for i,j in enumerate(top_empresas_ROA['ROA (%)']):
    ax.text(i,j+1,str(j) + '%',color='black',fontsize=12,ha='center',va='center')

ax.tick_params(left=False, bottom=False)


plt.show()

**Informe del Gráfico: Rentabilidad de activos en Empresas Americanas de la industria petrolera en el periodo 2015-2022**

Este gráfico de barras presenta las empresas americanas de la industria petrolera que tuvieron una rentabilidad de activos (ROA) superior al 20% en promedio durante el periodo 2015-2022. Weatherford International y Whitecap Resources destacan con un ROA promedio superior a este umbral, indicando una gestión eficiente de sus activos para generar ganancias.

In [ ]:
empresas_EU_top_rentabilidad

In [ ]:
plt.figure(figsize=(15, 8))

ax = sns.barplot(x='Margen_de_rentabilidad (%)', y='Industria', hue='Empresa', data=empresas_EU_top_rentabilidad, orient='h', palette=colores_personalizados)

plt.suptitle('Margen de Rentabilidad de las Empresas por Industria en Europa\n durante el periodo 2015-2022', fontsize=18, y=1.01)
plt.title('Empresas con mayor margen de rentabilidad por industria', fontsize=16)
plt.xlabel('Margen_de_rentabilidad (%)', fontsize=14)
plt.ylabel('Industria', fontsize=14)
plt.yticks(fontsize=12)

ax.set_xticklabels([])
sns.despine(left=True,bottom=True)
ax.legend(bbox_to_anchor=(1.2,-0.1), bbox_transform=ax.transAxes,
          fontsize=10, loc='lower right', title='Empresa')

for i,j in enumerate(empresas_EU_top_rentabilidad['Margen_de_rentabilidad (%)']):
    ax.text(j+3,i,str(j) + '%',color='black',fontsize=12,ha='center',va='center')

ax.tick_params(left=False, bottom=False)

plt.show()

**Informe del Gráfico: Margen de Rentabilidad de las Empresas por Industria en Europa durante el periodo 2015-2022**

Este gráfico de barras horizontal exhibe el margen de rentabilidad de las empresas europeas con el mayor porcentaje de rentabilidad por industria durante el periodo 2015-2022. Se observa la diversidad de industrias y las empresas líderes en rentabilidad dentro de cada sector en Europa a lo largo de estos años.

In [ ]:
empresas_NA_top_perdidas

In [ ]:
plt.figure(figsize=(15, 8))

ax = sns.barplot(x='Empresa', y='Ganancias', hue='Industria', data=empresas_NA_top_perdidas, palette='inferno')

plt.suptitle('Margen de Perdidas de las Empresas Norteamericanas por Industria\n durante el periodo 2015-2022', fontsize=18, y=1.01)
plt.title('Top 10 empresas con mayores perdidas', fontsize=16)
plt.xlabel('Empresas', fontsize=14)
plt.ylabel('Perdidas', fontsize=14)
plt.xticks(fontsize=12)
ax.set_yticklabels([])
sns.despine(left=True,bottom=True)
ax.legend(bbox_to_anchor=(1,0.1), bbox_transform=ax.transAxes,
          fontsize=10, loc='lower right', title='Industria')

for i,j in enumerate(empresas_NA_top_perdidas['Ganancias']):
    ax.text(i,j-500,str(j) + '$',color='black',fontsize=12,ha='center',va='center')

ax.tick_params(left=False, bottom=False)


plt.show()

**Informe del Gráfico: Margen de Perdidas de las Empresas Norteamericanas por Industria durante el periodo 2015-2022**

Este gráfico de barras muestra las 10 empresas norteamericanas con las mayores pérdidas registradas por industria a nivel global durante el periodo 2015-2022. Empresas en sectores como Oil & Gas Operations, Conglomerates y Transportation figuran entre las que experimentaron mayores pérdidas en este periodo.

In [ ]:
bancos_dist_activos_ingresos_15_22

In [ ]:
plt.figure(figsize=(15, 8))

ax = sns.scatterplot(x='Ingresos', y='Activos', data=bancos_dist_activos_ingresos_15_22, size='Ganancias', hue='Ganancias', sizes=(50, 200), palette='inferno_r')

plt.suptitle('Distribución de Ingresos y Activos con respecto a las ganancias de los bancos (2022)', fontsize=18, y=1.02)
plt.title('Distribución de Ingresos y Activos con respecto a las ganancias\n cuyos activos no superan los 150000 millones de dólares', fontsize=16)
plt.xlabel('Ingresos', fontsize=14)
plt.ylabel('Activos', fontsize=14)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
ax.legend(bbox_to_anchor=(1.2,-0.05), bbox_transform=ax.transAxes,
          fontsize=10, loc='lower right', title='Ganancias')

ax.tick_params(left=False, bottom=False)


plt.show()

**Informe del Gráfico: Distribución de Ingresos y Activos con respecto a las ganancias de los bancos (2015-2022)**

Este gráfico de dispersión muestra la distribución de Ingresos y Activos en relación con las ganancias de los bancos con activos que no superan los 15,000 millones de dólares durante el periodo 2015-2022. El tamaño de los puntos representa los ingresos. Permite visualizar la relación entre estas métricas para los bancos más pequeños a lo largo de los años.

##Extra

##Gráfico de carrera de barras: Variación del valor de mercado por industria durante el periodo 2015 - 2022

In [ ]:
df_forbes_2015_2022.head()

In [ ]:
df_valor_mercado_industria = df_forbes_2015_2022.groupby(['Industria','Ano'])[['Valor_de_mercado']].sum().reset_index()

display(df_valor_mercado_industria)

In [ ]:
df_valor_mercado_wide = df_valor_mercado_industria.pivot(index='Ano', columns='Industria', values='Valor_de_mercado')
display(df_valor_mercado_wide)

In [ ]:
# con esto llenamos los valores nulos con el primer o ultimo valor no nulo, segun convenga
# Esto para evitar columnas superpuestas, producto de como maneja la libreria las columnas
# con valores nulos

df_valor_mercado_wide.ffill(inplace=True)
df_valor_mercado_wide.bfill(inplace=True)

display(df_valor_mercado_wide)

In [ ]:
# df_valor_mercado_wide.to_csv('valor_mercado_por_industria.csv', index=False)

In [ ]:

warnings.filterwarnings('ignore')
# Guardamos el video en formato mp4 y lo mostramos en pantalla
bcr.bar_chart_race(df_valor_mercado_wide, 'valor_mercado_industria.mp4', fixed_max=True, steps_per_period=20, period_length=1000,
                   interpolate_period=True, figsize=(7,6), title='Valor de Mercado x Industria (2015-2019)', period_fmt='Año - {x:.0f}')

bcr.bar_chart_race(df_valor_mercado_wide, fixed_max=True, steps_per_period=20, period_length=1000,
                   interpolate_period=True, figsize=(7,6), title='Valor de Mercado x Industria (2015-2019)', period_fmt='Año - {x:.0f}')



**Informe del Gráfico: Carreras de Barras - Variación del valor de mercado por industria (2015-2019)**

Este gráfico de carreras de barras animado muestra la variación del valor de mercado total por industria en el ranking Forbes Global durante el periodo 2015-2019. Permite observar de forma dinámica cómo ha evolucionado el valor de mercado de las diferentes industrias a lo largo del tiempo, identificando las que han crecido o disminuido su valor.

In [ ]:
df_forbes_empleados.head()

In [ ]:
df_empleados_por_industria = df_forbes_empleados.groupby(['Ano','Industria'])[['Empleados']].sum().reset_index()

df_empleados_por_industria_wide = df_empleados_por_industria.pivot(index='Ano', columns='Industria', values='Empleados')

display(df_empleados_por_industria_wide)

In [ ]:
# con esto llenamos los valores nulos con el primer o ultimo valor no nulo, segun convenga
# Esto para evitar columnas superpuestas, producto de como maneja la libreria las columnas
# con valores nulos

df_empleados_por_industria_wide.ffill(inplace=True)
df_empleados_por_industria_wide.bfill(inplace=True)

df_empleados_por_industria_wide.info()

In [ ]:
# df_empleados_por_industria_wide.to_csv('empleados_por_industria.csv', index=False)

In [ ]:
# Guardamos el video en formato mp4 y lo mostramos en pantalla

bcr.bar_chart_race(df_empleados_por_industria_wide, 'Empleados_por_industria.mp4', fixed_max=True, steps_per_period=20, period_length=1000,
                   interpolate_period=True, figsize=(7,6), title='Numero de Empleados x Industria (2015-2019)',
                   period_fmt='Año - {x:.0f}')


bcr.bar_chart_race(df_empleados_por_industria_wide, fixed_max=True, steps_per_period=20, period_length=1000,
                   interpolate_period=True, figsize=(7,6), title='Numero de Empleados x Industria (2015-2019)',
                   period_fmt='Año - {x:.0f}')

**Informe del Gráfico: Carreras de Barras - Número de Empleados por Industria (2015-2019)**

Este gráfico de carreras de barras animado visualiza la variación en el número total de empleados por industria según el ranking Forbes Global durante el periodo 2015-2019. Este gráfico dinámico permite observar los cambios en el tamaño de la fuerza laboral en las diferentes industrias a lo largo del tiempo.

In [ ]:
df_forbes_2022.head()

In [ ]:
df_ventas_globales = df_forbes_2022.groupby(['Pais','Codigo'])[['Ingresos']].sum().reset_index()

display(df_ventas_globales.head())

In [ ]:
df_ventas_globales.to_csv('ventas_globales.csv',index=False)

In [ ]:
plt.figure(figsize=(15, 8))

fig = px.choropleth(df_ventas_globales, locations="Codigo",
                    color="Ingresos",
                    hover_name="Pais",
                    hover_data={'Ingresos':':.2f'},
                    color_continuous_scale=px.colors.sequential.Inferno,
                    title='Total de Ventas Globales x País (2022)')


fig.show()